In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import helper

import json
import os
import sys

In [2]:
course_schedule = []

IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20


RETRY_INTERVAL = 65

In [3]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element("xpath",xpath)

In [4]:
def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [5]:
def create_new_driver():
    #start web driver

    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service

    service = Service(executable_path=r'/usr/bin/chromedriver')

    #setup option for chrome profile
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=service, options=options)

    driver.implicitly_wait(IMPLICIT_WAIT_TIME)
    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    

    return driver, wait


In [6]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element("id",'ctl00_cphContents_txtUsername')
    password = driver.find_element("id",'ctl00_cphContents_txtPassword')
    
#     send_keys(username, "DN20092360")
#     send_keys(password, "jj1216")
    send_keys(username, "DN23035242")
    send_keys(password, "NTW3BSFH")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))

    sign_in_btn = driver.find_element(By.CLASS_NAME,"submit-button")
    sign_in_btn.click()

def log_out(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class():
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), '預定課')]")
#     print("click_btn: ", click_btn)
    click_btn.click()
    
def click_book_this_class_now(driver, wait):
    
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), '現在就預定此課')]")

    if click_btn:
        click_btn.click()
        return click_btn

In [7]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [8]:
temp = []

# load schedule from file

# with open("./class_schedules/30 十月 - 05 十一月 2023.json", "r") as read_file:
#     course_schedule = json.load(read_file)

with open("./class_schedules/06 十一月 - 12 十一月 2023.json", "r") as read_file:
    course_schedule = json.load(read_file)

Day_string = "12"

c = helper.search_courses_by_date(Day_string, course_schedule)
c = helper.search_courses_by_name('Gen', c)
c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('pm', c)

temp.extend(c)

courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 12 11月 周日
-----------------------

Name: Gentle Yoga 和緩瑜珈
Teacher: Una
Time: 09:45 - 10:45 上午




In [9]:
master_counter = 1

In [10]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [11]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [12]:
csv_file_name = "./log_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [13]:
class_idx = 1
num_classes = len(courses_to_be_booked)

In [14]:
def reserve_class(driver, wait, course_to_be_booked, idx):
    
    global master_counter
    global class_idx
    global num_classes

    stride =RETRY_INTERVAL / 60 * 15 * num_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    

    driver.get(book_url)

    click_btn=click_book_this_class_now(driver, wait)
    if click_btn is not None:
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',BOOKED,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        num_classes = num_classes - 1
    else:
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=RETRY_INTERVAL)
        log_str = "'{}','{}' by '{}',FULL,'{}'\nnext_book_datetime={}\nclick_btn={}\n".format(master_counter, class_name, teacher, datetime.datetime.now(), next_book_datetime, click_btn)
        print(log_str)
        
        if 0==0: #(master_counter - class_idx) % stride == 0:
            
            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
       
                
        driver.close()
        
        driver, wait = create_new_driver()

        sign_in_once(driver, wait)
        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60+(idx%len(course_to_be_booked))*10 )
        print("now={}".format(datetime.datetime.now()))
        print("next_book_datetime={}".format(next_book_datetime))
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked, idx])
    
        master_counter = master_counter + 1
    
    if class_idx < num_classes:
        class_idx = class_idx + 1
    else:
        class_idx = 1


In [15]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

#CSV file table header
with open(csv_file_name, "w") as write_file:
            write_file.write("#,Class,Status,Time\n")
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    
    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60+(idx%len(course_to_be_booked))*10 )
    print("now={}".format(datetime.datetime.now()))
    print("next_book_datetime={}".format(next_book_datetime))
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked, idx])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

now=2023-11-12 00:28:18.971420
next_book_datetime=2023-11-12 00:29:18.971392
master counter = 1, stride = 16.25, class_idx = 1
'1','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 00:29:39.627138'
next_book_datetime=2023-11-12 00:30:44.627115
click_btn=None

now=2023-11-12 00:29:43.168128
next_book_datetime=2023-11-12 00:30:43.168103
master counter = 2, stride = 16.25, class_idx = 1
'2','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 00:31:03.818203'
next_book_datetime=2023-11-12 00:32:08.818171
click_btn=None

now=2023-11-12 00:31:07.401113
next_book_datetime=2023-11-12 00:32:07.401082
master counter = 3, stride = 16.25, class_idx = 1
'3','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 00:32:28.058473'
next_book_datetime=2023-11-12 00:33:33.058446
click_btn=None

now=2023-11-12 00:32:31.583162
next_book_datetime=2023-11-12 00:33:31.583132
master counter = 4, stride = 16.25, class_idx = 1
'4','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 00:33:52.232924'
next_book_datetime=2023-11-12 00:34:57.232

now=2023-11-12 01:13:09.428609
next_book_datetime=2023-11-12 01:14:09.428585
master counter = 33, stride = 16.25, class_idx = 1
'33','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 01:14:30.104653'
next_book_datetime=2023-11-12 01:15:35.104624
click_btn=None

now=2023-11-12 01:14:33.540136
next_book_datetime=2023-11-12 01:15:33.540110
master counter = 34, stride = 16.25, class_idx = 1
'34','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 01:15:54.211629'
next_book_datetime=2023-11-12 01:16:59.211605
click_btn=None

now=2023-11-12 01:15:57.488323
next_book_datetime=2023-11-12 01:16:57.488299
master counter = 35, stride = 16.25, class_idx = 1
'35','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 01:17:18.174636'
next_book_datetime=2023-11-12 01:18:23.174610
click_btn=None

now=2023-11-12 01:17:21.520960
next_book_datetime=2023-11-12 01:18:21.520934
master counter = 36, stride = 16.25, class_idx = 1
'36','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 01:18:42.216744'
next_book_datetime=2023-11-12 01:1

now=2023-11-12 01:58:11.741377
next_book_datetime=2023-11-12 01:59:11.741351
master counter = 65, stride = 16.25, class_idx = 1
'65','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 01:59:32.387385'
next_book_datetime=2023-11-12 02:00:37.387361
click_btn=None

now=2023-11-12 01:59:36.105568
next_book_datetime=2023-11-12 02:00:36.105539
master counter = 66, stride = 16.25, class_idx = 1
'66','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 02:00:56.766796'
next_book_datetime=2023-11-12 02:02:01.766772
click_btn=None

now=2023-11-12 02:01:00.265438
next_book_datetime=2023-11-12 02:02:00.265411
master counter = 67, stride = 16.25, class_idx = 1
'67','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 02:02:20.999016'
next_book_datetime=2023-11-12 02:03:25.998993
click_btn=None

now=2023-11-12 02:02:24.693456
next_book_datetime=2023-11-12 02:03:24.693429
master counter = 68, stride = 16.25, class_idx = 1
'68','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 02:03:45.341494'
next_book_datetime=2023-11-12 02:0

now=2023-11-12 02:43:04.143819
next_book_datetime=2023-11-12 02:44:04.143792
master counter = 97, stride = 16.25, class_idx = 1
'97','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 02:44:24.815408'
next_book_datetime=2023-11-12 02:45:29.815384
click_btn=None

now=2023-11-12 02:44:28.384259
next_book_datetime=2023-11-12 02:45:28.384227
master counter = 98, stride = 16.25, class_idx = 1
'98','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 02:45:49.227538'
next_book_datetime=2023-11-12 02:46:54.227482
click_btn=None

now=2023-11-12 02:45:52.664826
next_book_datetime=2023-11-12 02:46:52.664795
master counter = 99, stride = 16.25, class_idx = 1
'99','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 02:47:13.360771'
next_book_datetime=2023-11-12 02:48:18.360745
click_btn=None

now=2023-11-12 02:47:16.795938
next_book_datetime=2023-11-12 02:48:16.795909
master counter = 100, stride = 16.25, class_idx = 1
'100','Gentle Yoga 和緩瑜珈' by 'Una',FULL,'2023-11-12 02:48:37.477215'
next_book_datetime=2023-11-12 02